# pattern for simple amendments to annex

In [1]:
#table annex with numbers
import pandas
import os
import requests
import re
from bs4 import BeautifulSoup

url = 'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32022R0340'
response = requests.get(url)
print(response.status_code)
data = response.text
data

200


'<?xml version="1.0" encoding="UTF-8"?>\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML//EN" "xhtml-strict.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<!-- CONVEX # converter_version:9.15.0 # generated_on:20240314-0520 # ELI version:0.10 -->\r\n<head><meta name="format-detection" content="telephone=no"/><meta http-equiv=\'X-UA-Compatible\' content=\'IE=edge\'/><meta name=\'viewport\' content=\'width=device-width, initial-scale=1\'><meta name=\'WT.z_usr_lan\' content=\'EN\'>\n   <meta http-equiv="content-type" content="text/html; charset=utf-8"/>\n   <script type="text/javascript" src="/ruxitagentjs_ICANVfgqrux_10289240325103055.js" data-dtconfig="app=47d4c64c3b67ec69|agentId=2801507a3763167f|featureHash=ICANVfgqrux|rdnt=1|uxrgce=1|bp=3|cuc=m097nmfl|mel=100000|mb=null|dpvc=1|iub=null|ssv=4|lastModification=1720772123244|tp=500,50,0|agentUri=/ruxitagentjs_ICANVfgqrux_10289240325103055.js|reportUrl=/rb_39a3e95b-5423-482c-879b-99ef235dffeb|rid=RID_2079675099|rpid=1903599457|domain=

In [49]:
#get the document information

url= re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data)
publication_date = re.findall(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data)
title = re.findall(r'class="oj-doc-ti"[^>]*>([\s\S]*?)<\/p>', data)
celex= re.findall(r'uri=CELEX%([^"]+)', data)
print(url),print(publication_date),print(title),print(celex)


['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32022R0340']
['28.2.2022']
['\n            COUNCIL IMPLEMENTING REGULATION (EU)\xa02022/340', 'of 28\xa0February 2022\n         ', 'implementing Article\xa012 of Regulation (EU) No\xa0356/2010 imposing certain specific restrictive measures directed against certain natural or legal persons, entities or bodies, in view of the situation in Somalia', 'ANNEX']
['3A32022R0340']


(None, None, None, None)

In [33]:
legal_base = re.findall(r'\((CFSP|EU)\)\s*(\d+/\d+)', data)
legal_base

[('EU', '2022/340')]

In [31]:
#search for UN to add the UN dummy indicator
UN_pattern = re.search(r'UN',data)

UN = None
for i in data:
    if UN_pattern == None:
        UN = 0
    else:
        UN = 1

In [76]:
name = re.search(r'class="oj-bold">\d+\.\s*(.*?)\s*\(alias:\s*(.*?)\)</span>', data)
print(name)
#alias = 

None


In [66]:
# Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')

# List to store the parsed data
sanction_list = []
name = re.findall(r'class="oj-bold">\d+\.(.*?)</p>',data)
    #identifying_info = cells[2].get_text(strip=True)  # Extracting the identifying information
    #reasons = ' '.join(p.get_text(strip=True) for p in cells[3].find_all('p'))
    #date_of_listing = cells[4].get_text(strip=True)
    
sanction_list.append({
        'URL': url,
        'CELEX':celex,
        'law_number' :title[0],
        'Publication Date': publication_date,
        'legal_base' : title[2],
        'Name': name,
        'DOB': re.findall(r'>Date of Birth</span>(.*?)</p>',data),
        'POB': re.findall(r'Place of Birth</span>:(.*?)</p>',person_info),
        'adress':  re.findall(r'Adress:</span>(.*?)</p>',person_info),
        'nationality':  re.findall(r'Nationality:</span>(.*?)</p>',person_info),
        'Identifying Information':  re.findall(r'Other Information:</span>(.*?)</p>',person_info),
        'Reason':  re.findall(r'Designation:</span>(.*?)</p>',person_info),
        'UN_sanction_date' :  re.findall(r'Date of UN designation:</span>(.*?)</p>',person_info),
        'Date of Listing': publication_date,
        'UN_sanctions_dummy' : UN
    })

# Display the results
for person in sanction_list:
    print(person)


{'URL': ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32022R0340'], 'CELEX': ['3A32022R0340'], 'law_number': '\n            COUNCIL IMPLEMENTING REGULATION (EU)\xa02022/340', 'Publication Date': ['28.2.2022'], 'legal_base': 'implementing Article\xa012 of Regulation (EU) No\xa0356/2010 imposing certain specific restrictive measures directed against certain natural or legal persons, entities or bodies, in view of the situation in Somalia', 'Name': [], 'DOB': [': 1966'], 'POB': [], 'adress': [], 'nationality': [], 'Identifying Information': [], 'Reason': [], 'UN_sanction_date': [], 'Date of Listing': ['28.2.2022'], 'UN_sanctions_dummy': 1}


In [59]:
tbody

<tbody>
<tr>
<td>
<p class="oj-hd-date">28.2.2022   </p>
</td>
<td>
<p class="oj-hd-lg">EN</p>
</td>
<td>
<p class="oj-hd-ti">Official Journal of the European Union</p>
</td>
<td>
<p class="oj-hd-oj">L 56/1</p>
</td>
</tr>
</tbody>

In [ ]:
# Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')

# List to store the parsed data
sanction_list = []

# Find all rows in the table
rows = soup.find_all('tr')

# Iterate through the rows and extract the necessary information
for row in rows:
    cells = row.find_all('td')
    if len(cells) < 2:
        continue

    # Extract information using regex
    person_info = cells[1].get_text(separator=' ', strip=True)
    
    name = re.findall(r'[\d]+\.\s+(.*?)(?:\(|$)', person_info)
    aliases = re.findall(r'alias:\s+(.*?)\)', person_info)
    designation = re.findall(r'Designation:\s+(.*?)\s', person_info)
    dob = re.findall(r'Date of Birth:\s+(.*?)\s', person_info)
    pob = re.findall(r'Place of Birth:\s+(.*?)\s', person_info)
    address = re.findall(r'Address:\s+(.*?)\s', person_info)
    nationality = re.findall(r'Nationality:\s+(.*?)\s', person_info)
    un_designation_date = re.findall(r'Date of UN designation:\s+(.*?)\s', person_info)
    reason = re.findall(r'Other information:\s+(.*)', person_info)

    sanction_list.append({
        'URL': url,
        'CELEX': celex,
        'law_number': title[0],
        'Publication Date': publication_date,
        'legal_base': title[2],
        'Name': name[0] if name else '',
        'DOB': dob[0] if dob else '',
        'POB': pob[0] if pob else '',
        'address': address[0] if address else '',
        'nationality': nationality[0] if nationality else '',
        'Identifying Information': '',
        'Reason': reason[0].strip() if reason else '',
        'UN_sanction_date': un_designation_date[0] if un_designation_date else '',
        'Date of Listing': publication_date,
        'UN_sanctions_dummy': UN
    })

# Display the results
for person in sanction_list:
    print(person)

{'URL': 'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32022R0340', 'CELEX': ['3A32022R0340'], 'law_number': 'A', 'Publication Date': '28.2.2022', 'legal_base': 'N', 'Name': '', 'DOB': '', 'POB': '', 'address': '', 'nationality': '', 'Identifying Information': '', 'Reason': '', 'UN_sanction_date': '', 'Date of Listing': '28.2.2022', 'UN_sanctions_dummy': 1}
{'URL': 'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32022R0340', 'CELEX': ['3A32022R0340'], 'law_number': 'A', 'Publication Date': '28.2.2022', 'legal_base': 'N', 'Name': '', 'DOB': '', 'POB': '', 'address': '', 'nationality': '', 'Identifying Information': '', 'Reason': '', 'UN_sanction_date': '', 'Date of Listing': '28.2.2022', 'UN_sanctions_dummy': 1}
{'URL': 'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32022R0340', 'CELEX': ['3A32022R0340'], 'law_number': 'A', 'Publication Date': '28.2.2022', 'legal_base': 'N', 'Name': '', 'DOB': '', 'POB': '', 'address': '', 'nationa

In [41]:
sanction_list

[]

In [55]:
from bs4 import BeautifulSoup
import re

soup = BeautifulSoup(data, 'html.parser')

# List to store the parsed data
sanction_list = []

# Find the tbody containing the relevant person entries
tbody = soup.find('tbody')

# Iterate through the rows within the tbody and extract the necessary information
for row in tbody.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) < 2:
        continue

    # Extract information using regex
    person_info = cells[1].get_text(separator=' ', strip=True)
    
    name = re.findall(r'[\d]+\.\s+(.*?)(?:\(|$)', person_info)
    aliases = re.findall(r'alias:\s+(.*?)\)', person_info)
    designation = re.findall(r'Designation:\s+(.*?)\s', person_info)
    dob = re.findall(r'Date of Birth:\s+(.*?)\s', person_info)
    pob = re.findall(r'Place of Birth:\s+(.*?)\s', person_info)
    address = re.findall(r'Address:\s+(.*?)\s', person_info)
    nationality = re.findall(r'Nationality:\s+(.*?)\s', person_info)
    un_designation_date = re.findall(r'Date of UN designation:\s+(.*?)\s', person_info)
    reason = re.findall(r'Other information:\s+(.*)', person_info)

    sanction_list.append({
        'URL': url,
        'CELEX': celex,
        'law_number': title[0],
        'Publication Date': publication_date,
        'legal_base': title[2],
        'Name': name[0] if name else '',
        'DOB': dob[0] if dob else '',
        'POB': pob[0] if pob else '',
        'address': address[0] if address else '',
        'nationality': nationality[0] if nationality else '',
        'Identifying Information': '',
        'Reason': reason[0].strip() if reason else '',
        'UN_sanction_date': un_designation_date[0] if un_designation_date else '',
        'Date of Listing': publication_date,
        'UN_sanctions_dummy': UN
    })

# Display the results
for person in sanction_list:
    print(person)



{'URL': ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32022R0340'], 'CELEX': ['3A32022R0340'], 'law_number': '\n            COUNCIL IMPLEMENTING REGULATION (EU)\xa02022/340', 'Publication Date': ['28.2.2022'], 'legal_base': 'implementing Article\xa012 of Regulation (EU) No\xa0356/2010 imposing certain specific restrictive measures directed against certain natural or legal persons, entities or bodies, in view of the situation in Somalia', 'Name': '', 'DOB': '', 'POB': '', 'address': '', 'nationality': '', 'Identifying Information': '', 'Reason': '', 'UN_sanction_date': '', 'Date of Listing': ['28.2.2022'], 'UN_sanctions_dummy': 1}
